### 照片拍攝，透過opencv拍照

In [ ]:
import cv2

cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)

num = 0

while cap.isOpened():

    succes, img = cap.read()

    k = cv2.waitKey(5)

    if k == ord("q"):
        break
    elif k == ord('s'): # wait for 's' key to save and exit
        cv2.imwrite('images/img' + str(num) + '.png', img)
        print("image saved!")
        num += 1

    cv2.imshow('Img',img)

cap.release()

cv2.destroyAllWindows()

### 繪圖函數宣告

In [ ]:
# 繪製函數定義
import numpy as np
def draw_axis(img, corners, image_points):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(np.int16(image_points[0].ravel())), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(np.int16(image_points[1].ravel())), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(np.int16(image_points[2].ravel())), (0,0,255), 5)
    return img
def draw_xy_lines(img, image_points):
    img = cv2.line(img, tuple(np.int16(image_points[0].ravel())), tuple(np.int16(image_points[1].ravel())), (0,0,0), 5)
    img = cv2.line(img, tuple(np.int16(image_points[1].ravel())), tuple(np.int16(image_points[2].ravel())), (0,0,0), 5)
    return img
def draw_z_lines(img, image_points):
    img = cv2.line(img, tuple(np.int16(image_points[1].ravel())), tuple(np.int16(image_points[3].ravel())), (128,128,0), 5)
    return img

# 方塊座標系轉換至桌面座標系

模型讀取

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import cube_detector.cube_detector as CD
import pickle

model = YOLO("yolov8n-seg-custom.pt")
surface_model = YOLO('cube_surface_seg2.pt')


方塊到桌面座標系轉換

### 照片計算

In [ ]:
detector=CD.CubeDetector(model,surface_model)
detector
with open('test_calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)
with open('test_table_points.pkl','rb') as file:
    table_objectPoints,table_imagePoints = pickle.load(file)
with open('test_table_solvePnP.pkl','rb') as file:
    table_rvec,table_tvec = pickle.load(file)
color = "green"
img = cv2.imread("test.jpg")
height =480
scale = height / img.shape[0]
img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
#  將圖片丟入偵測器進行偵測
result_img = detector.detect(img,index=None,color=None)
# 讀取抓到的角點座標
cube_imagePoints=detector.get_cube_sequence_imagePoints(color)
# 依順序定出對應真實世界對應座標
cube_objectPoints=np.array([(0,0,0),(0,25,0),(25,25,0),(25,0,0),(25,0,-25),(0,0,-25)])
# 如果抓到六個點採用EPNP算法(較準確穩定)，四個點則是一般算法
if cube_imagePoints.shape[0]==6:
    PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints.astype(float),cube_imagePoints,mtx,dist,flags=cv2.SOLVEPNP_EPNP)
elif cube_imagePoints.shape[0]==4:
    PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints[:4].astype(float),cube_imagePoints,mtx,dist)
#  若成功計算出來則計算出該方塊相對於桌面座標系的座標位置
if PnP_success:
    R1, _ = cv2.Rodrigues(cube_rvec) # 創建方塊到相機座標系的旋轉矩陣
    R2, _ = cv2.Rodrigues(table_rvec) # 創建桌面到方塊座標系的旋轉矩陣
    T1 = np.eye(4) #創建4*4單位矩陣
    T2 = np.eye(4) 
    T1[:3, :3], T1[:3, 3] = R1, cube_tvec.T #  方塊座標系旋轉加平移到相機座標系 轉換矩陣
    T2[:3, :3], T2[:3, 3] = R2, table_tvec.T # 桌面座標系旋轉加平移到相機座標系 轉換矩陣
    T2=np.linalg.inv(T2) # 取逆矩陣，變成 相譏座標系到桌面座標系 轉換矩陣
    transform_matrix = T2 @ T1 # 結合二者，得到方塊座標系到桌面座標系的轉換矩陣
    transformed_point = transform_matrix @ np.array([[12.5, 12.5, -12.5, 1]]).T # 傳入方塊正中央座標，即可得到方塊在桌面座標位置了
    # 输出结果
    print("物體在桌面座標系下的座標:\n", transformed_point)
# xyz座標位置線繪製
x,y,z = transformed_point[0:3].flatten()
xyz_line_points=np.float32([[0,y,0],[x,y,0],[x,0,0],[x,y,z]]).reshape(-1,3)
xyz_line_points_img,_=cv2.projectPoints(xyz_line_points,table_rvec,table_tvec,mtx,dist)
result_img = draw_xy_lines(result_img,xyz_line_points_img)
result_img = draw_z_lines(result_img,xyz_line_points_img)

# xyz座標軸繪製
axis = np.float32([[25,0,0], [0,25,0], [0,0,25]]).reshape(-1,3)
axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
result_img=draw_axis(result_img,cube_imagePoints.astype(int),axis_cube_img_points)
result_img=draw_axis(result_img,table_imagePoints.astype(int),axis_table_img_points)

# 取得方塊輪廓計算方塊重心位置
contour_outer = detector.get_cube_contour_outer(color)
radius = int(0.04*cv2.arcLength(contour_outer,True))
print(radius)
M = cv2.moments(contour_outer)
if M["m00"] != 0:
    centroid_X = int(M["m10"] / M["m00"])# 算形心x
    centroid_Y = int(M["m01"] / M["m00"])# 算形心y
# 繪製重心座標點
result_img = cv2.circle(result_img, (centroid_X,centroid_Y),radius, (255,0,255), 2)

# 繪製PNP計算所得座標點
predict_cube_center_imagePoint = cv2.projectPoints(np.float32([x,y,z]),table_rvec,table_tvec,mtx,dist)
predict_cube_center_imagePoint = predict_cube_center_imagePoint[0].astype(np.int16)

cv2.putText(result_img, f"{x=:.1f}, {y=:.1f}, {z=:.1f}", (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (128, 0, 128), 2)

# 推算是否位置估計錯誤
if np.linalg.norm(np.array((centroid_X,centroid_Y))-predict_cube_center_imagePoint)>radius:
    cv2.putText(result_img, f"Error", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
    result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(),5, (0,0,255), -1)
else:
    cv2.putText(result_img, f"Ok", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
    result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(),5, (0,255,0), -1)
cv2.imshow("result",result_img)
cv2.waitKey(0)

    

### 即時偵測主程式

In [ ]:
hand_cam = cv2.VideoCapture(0,cv2.CAP_DSHOW)
fixed_cam = cv2.VideoCapture(1,cv2.CAP_DSHOW)

In [ ]:
hand_cam.release()
fixed_cam.release()

In [ ]:
import importlib

importlib.reload(CD)

In [28]:
detector = CD.CubeDetector(model, surface_model)
cap = hand_cam
with open("test_calibration.pkl", "rb") as file:
    mtx, dist = pickle.load(file)
# with open('test_table_points.pkl','rb') as file:
#     table_objectPoints,table_imagePoints = pickle.load(file)
# with open('test_table_solvePnP.pkl','rb') as file:
#     table_rvec,table_tvec = pickle.load(file)
fixed_to_table = np.float32(
    [[0, 1, 0, 220],
     [1, 0, 0, 540],
     [0, 0, -1, 860],
     [0, 0, 0, 1]]
)
_, table_rmtx, table_tvec, _, _, _, _  = cv2.decomposeProjectionMatrix(fixed_to_table[0:3,0:])

table_rvec, _ = cv2.Rodrigues(table_rmtx)
table_tvec = fixed_to_table[0:3,3]
print(table_tvec)

color = "green"
while cap.isOpened():
    cap_success, frame = cap.read()
    if cap_success:
        height = 480
        scale = height / frame.shape[0]
        frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        #  將圖片丟入偵測器進行偵測
        result_img = detector.detect(frame, index=None, color=None)
        # 讀取抓到的角點座標
        cube_imagePoints = detector.get_cube_sequence_imagePoints(color)
        if cube_imagePoints is None:
            cv2.imshow("result", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            continue
        # 依順序定出對應真實世界對應座標
        cube_objectPoints = np.array(
            [(0, 0, 0), (0, 25, 0), (25, 25, 0),
             (25, 0, 0), (25, 0, -25), (0, 0, -25)]
        )
        # 如果抓到六個點採用EPNP算法(較準確穩定)，四個點則是一般算法
        if cube_imagePoints.shape[0] == 6:
            PnP_success, cube_rvec, cube_tvec = cv2.solvePnP(
                cube_objectPoints.astype(float), cube_imagePoints, mtx, dist, flags=cv2.SOLVEPNP_EPNP)
        elif cube_imagePoints.shape[0] == 4:
            PnP_success, cube_rvec, cube_tvec = cv2.solvePnP(
                cube_objectPoints[:4].astype(float), cube_imagePoints, mtx, dist)
        #  若成功計算出來則計算出該方塊相對於桌面座標系的座標位置
        print(cube_tvec)
        print(cube_rvec)
        if PnP_success:
            R1, _ = cv2.Rodrigues(cube_rvec)  # 創建方塊到相機座標系的旋轉矩陣
            R2, _ = cv2.Rodrigues(table_rvec)  # 創建桌面到方塊座標系的旋轉矩陣
            T1 = np.eye(4)  # 創建4*4單位矩陣
            T2 = np.eye(4)
            T1[:3, :3], T1[:3, 3] = R1, cube_tvec.T  #  方塊座標系旋轉後平移到相機座標系 轉換矩陣
            T2[:3, :3], T2[:3, 3] = R2, table_tvec.T  # 桌面座標系旋轉後平移到相機座標系 轉換矩陣
            # T2 = np.linalg.inv(T2)  # 取逆矩陣，變成 相譏座標系到桌面座標系 轉換矩陣
            # fixed_to_table = np.linalg.inv(fixed_to_table)  # 取逆矩陣，變成 相譏座標系到桌面座標系 轉換矩陣
            transform_matrix = fixed_to_table @ T1  # 結合二者，得到方塊座標系到桌面座標系的轉換矩陣
            transformed_point = (transform_matrix @ np.array([[12.5, 12.5, -12.5, 1]]).T)  # 傳入方塊正中央座標，即可得到方塊在桌面座標位置了
            # 输出结果
            print("物體在桌面座標系下的座標:\n", transformed_point)
        # xyz座標位置線繪製
        x, y, z = transformed_point[0:3].flatten()
        xyz_line_points = np.float32([[0, y, 0], [x, y, 0], [x, 0, 0], [x, y, z]]).reshape(-1, 3)
        xyz_line_points_img, _ = cv2.projectPoints(xyz_line_points, table_rvec, table_tvec, mtx, dist)
        result_img = draw_xy_lines(result_img, xyz_line_points_img)
        result_img = draw_z_lines(result_img, xyz_line_points_img)

        # xyz座標軸繪製
        axis = np.float32([[25, 0, 0], [0, 25, 0], [0, 0, 25]]).reshape(-1, 3)
        axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
        axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
        result_img = draw_axis(result_img, cube_imagePoints.astype(int), axis_cube_img_points)
        result_img = draw_axis(result_img, table_imagePoints.astype(int), axis_table_img_points)

        # 取得方塊輪廓計算方塊重心位置
        contour_outer = detector.get_cube_contour_outer(color)
        radius = int(0.04 * cv2.arcLength(contour_outer, True))
        print(radius)
        M = cv2.moments(contour_outer)
        if M["m00"] != 0:
            centroid_X = int(M["m10"] / M["m00"])  # 算形心x
            centroid_Y = int(M["m01"] / M["m00"])  # 算形心y
        # 繪製重心座標點
        result_img = cv2.circle(result_img, (centroid_X, centroid_Y), radius, (255, 0, 255), 2)

        # 繪製PNP計算所得座標點
        predict_cube_center_imagePoint = cv2.projectPoints(np.float32([x, y, z]), table_rvec, table_tvec, mtx, dist)
        predict_cube_center_imagePoint = predict_cube_center_imagePoint[0].astype(np.int16)

        cv2.putText(result_img, f"{x=:.1f}, {y=:.1f}, {z=:.1f}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (128, 0, 128), 2,)

        # 推算是否位置估計錯誤
        if (np.linalg.norm(np.array((centroid_X, centroid_Y)) - predict_cube_center_imagePoint) > radius):
            cv2.putText(result_img, f"Error", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4,)
            result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(), 5, (0, 0, 255), -1)
        else:
            cv2.putText(result_img, f"Ok", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4,)
            result_img = cv2.circle(
                result_img, predict_cube_center_imagePoint.ravel(), 5, (0, 255, 0), -1)
        cv2.imshow("result", result_img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break
cv2.destroyAllWindows()

[        220         540         860]
[[    -26.559]
 [    -27.323]
 [     488.17]]
[[   0.014413]
 [    -2.8166]
 [    -1.0347]]
物體在桌面座標系下的座標:
 [[     193.46]
 [     503.25]
 [     352.74]
 [          1]]
8
[面]輪廓抓取近似多邊形失敗
[[    -26.304]
 [    -26.862]
 [     478.56]]
[[   0.019666]
 [    -2.8147]
 [    -1.0187]]
物體在桌面座標系下的座標:
 [[     194.03]
 [      503.6]
 [     362.31]
 [          1]]
8
[面]輪廓抓取近似多邊形失敗
[[    -25.829]
 [    -26.849]
 [     475.14]]
[[   0.031138]
 [    -2.8317]
 [    -1.0093]]
物體在桌面座標系下的座標:
 [[     194.18]
 [     503.78]
 [     365.89]
 [          1]]
8
[面]輪廓抓取近似多邊形失敗
[[    -25.874]
 [    -26.611]
 [      473.5]]
[[   0.027437]
 [    -2.8191]
 [    -1.0112]]
物體在桌面座標系下的座標:
 [[     194.34]
 [     503.96]
 [     367.41]
 [          1]]
8
[面]輪廓抓取近似多邊形失敗
[[    -26.304]
 [    -26.862]
 [     478.56]]
[[   0.019666]
 [    -2.8147]
 [    -1.0187]]
物體在桌面座標系下的座標:
 [[     194.03]
 [      503.6]
 [     362.31]
 [          1]]
8
[面]輪廓抓取近似多邊形失敗
[[    -25.739]
 [    -26.996]
 [     

C:\Users\user\AppData\Local\Temp\ipykernel_10916\3344818570.py:14: RuntimeWarning: invalid value encountered in cast
  img = cv2.line(img, tuple(np.int16(image_points[1].ravel())), tuple(np.int16(image_points[3].ravel())), (128,128,0), 5)
C:\Users\user\AppData\Local\Temp\ipykernel_10916\1739996930.py:92: RuntimeWarning: invalid value encountered in cast
  predict_cube_center_imagePoint = predict_cube_center_imagePoint[0].astype(np.int16)


[[    -14.349]
 [    -56.292]
 [     435.76]]
[[    0.37751]
 [    -2.1055]
 [   -0.50667]]
物體在桌面座標系下的座標:
 [[     166.33]
 [     528.44]
 [     402.93]
 [          1]]
9
[[    -16.205]
 [    -60.179]
 [     467.91]]
[[     1.9702]
 [     1.8643]
 [     1.2496]]
物體在桌面座標系下的座標:
 [[     162.98]
 [     523.62]
 [     370.68]
 [          1]]
9
[[    -15.721]
 [    -58.148]
 [     472.13]]
[[     2.0182]
 [     1.7431]
 [     1.3505]]
物體在桌面座標系下的座標:
 [[     163.32]
 [     523.49]
 [     366.29]
 [          1]]
9
[[    -17.381]
 [    -58.792]
 [     496.28]]
[[    0.44895]
 [    -2.3542]
 [   -0.77416]]
物體在桌面座標系下的座標:
 [[     159.62]
 [     519.71]
 [     342.33]
 [          1]]
9
[[    -2.3411]
 [    -58.331]
 [     470.35]]
[[   0.089788]
 [     2.2462]
 [    0.45452]]
物體在桌面座標系下的座標:
 [[     172.31]
 [     519.02]
 [     386.82]
 [          1]]
9
[[    -20.092]
 [    -44.127]
 [     437.64]]
[[      2.087]
 [     1.5739]
 [     1.4892]]
物體在桌面座標系下的座標:
 [[     174.86]
 [     518.29]
 [     400.79

In [17]:
import pickle
import numpy as np
# with open('test_calibration.pkl', 'rb') as file:
#     mtx,dist = pickle.load(file)
#     print(mtx,dist)
with open('./hand_matrix/camMatrix.npy', 'rb') as file:
    mtx = np.load(file)
    print(mtx)
# with open('./Hcam2Grip/Hcam2grip.npz', 'rb') as file:
#     mtx = np.load(file)
#     for item in mtx.files:
#         print(item)
#         print(mtx[item])


[[      638.8           0      298.64]
 [          0      638.68      236.92]
 [          0           0           1]]


In [ ]:
import numpy as np
m1 = np.array(
    [[1, 0, 0, 1],
     [0, 1, 0, 2],
     [0, 0, 1, 3],
     [0, 0, 0, 1]])
m2 = np.array(
    [[0, 1, 0, 0],
     [1, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 1]])
m3 = np.array(
    [[2],
     [3],
     [1],
     [1]])
m1@m2